## Capital Bikeshare Data Processing 

In [1]:
#Importing packages
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set(style="darkgrid")
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

In [3]:
#Working Directory
import os
#cd
#os.getcwd()
os.chdir('D:\\Users\\RWang\\Box Sync\\Inclusion&Innovation\\Data Collection and Process\\Capital Bikeshare\\2017-capitalbikeshare-tripdata')
os.getcwd()

'D:\\Users\\RWang\\Box Sync\\Inclusion&Innovation\\Data Collection and Process\\Capital Bikeshare\\2017-capitalbikeshare-tripdata'

#### 2017 Quarterly Trip Data Files 

In [26]:
##Access bike data files
import glob
path = os.getcwd()
all_files = glob.glob(path + "/*.csv")

frame = pd.DataFrame()
list_ = []

for file_ in all_files:
    df = pd.read_csv(file_,index_col=None,header=0)
    list_.append(df)
frame = pd.concat(list_)

In [54]:
#Initial data examination
if str(type(frame)) == "<class 'pandas.core.frame.DataFrame'>": 
    print(frame.shape)
else:
    print("Check Code")

frame.head(10)

(3757777, 9)


,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,221,2017-01-01 00:00:41,2017-01-01 00:04:23,31634,3rd & Tingey St SE,31208,M St & New Jersey Ave SE,W00869,Member
1,1676,2017-01-01 00:06:53,2017-01-01 00:34:49,31258,Lincoln Memorial,31270,8th & D St NW,W00894,Casual
2,1356,2017-01-01 00:07:10,2017-01-01 00:29:47,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,31222,New York Ave & 15th St NW,W21945,Casual
3,1327,2017-01-01 00:07:22,2017-01-01 00:29:30,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,31222,New York Ave & 15th St NW,W20012,Casual
4,1636,2017-01-01 00:07:36,2017-01-01 00:34:52,31258,Lincoln Memorial,31270,8th & D St NW,W22786,Casual
5,1603,2017-01-01 00:08:11,2017-01-01 00:34:55,31258,Lincoln Memorial,31270,8th & D St NW,W20890,Casual
6,473,2017-01-01 00:08:36,2017-01-01 00:16:29,31611,13th & H St NE,31616,3rd & H St NE,W20340,Member
7,200,2017-01-01 00:11:07,2017-01-01 00:14:27,31104,Adams Mill & Columbia Rd NW,31121,Calvert St & Woodley Pl NW,W20398,Member
8,748,2017-01-01 00:13:20,2017-01-01 00:25:49,31041,Prince St & Union St,31097,Saint Asaph & Madison St,W00365,Member
9,912,2017-01-01 00:14:35,2017-01-01 00:29:48,31202,14th & R St NW,31505,Eckington Pl & Q St NE,W20771,Member


In [69]:
#Confirm data indeed merged from all four files
tempdf1 = pd.read_csv(all_files[0])
tempdf2 = pd.read_csv(all_files[1])
tempdf3 = pd.read_csv(all_files[2])
tempdf4 = pd.read_csv(all_files[3])
n_row = tempdf1.shape[0] + tempdf2.shape[0] + tempdf3.shape[0] + tempdf4.shape[0]
if (n_row == frame.shape[0]):
    print("PROCEED...")
    print("REMOVING: TEMP FILES")
    del tempdf1, tempdf2, tempdf3, tempdf4
else:
    print("PROBLEM: UNEQUAL ROW COUNT")

PROCEED...
REMOVING: TEMP FILES


In [78]:
frame.insert(0,'Index',range(1,len(frame)+1))
print(list(frame.columns.values))


['Index', 'Duration', 'Start date', 'End date', 'Start station number', 'Start station', 'End station number', 'End station', 'Bike number', 'Member type']


In [119]:
#GEOCODING...
    #print(list(frame.columns.values))
df_address = frame[['Index','Start station','End station']]
df_address = df_address.rename(columns={"Start station": "StartStation"})
df_address = df_address.rename(columns={"End station": "EndStation"})

(487,)

In [122]:
df_address['StartStation'].unique().shape
list_start = df_address['StartStation'].unique().tolist()

In [123]:
df_address['EndStation'].unique().shape
list_end = df_address['EndStation'].unique().tolist()

In [126]:
#Check all stations in start list also in end station list
if set(list_start) == set(list_end):
    list_stations = list_start
    del list_start, list_end
    print(list_stations)
else:
    print('CHECK SOURCE')
    

['3rd & Tingey St SE', 'Lincoln Memorial', 'Henry Bacon Dr & Lincoln Memorial Circle NW', '13th & H St NE', 'Adams Mill & Columbia Rd NW', 'Prince St & Union St', '14th & R St NW', 'Market Square / King St & Royal St', '3rd & H St NE', 'Lamont & Mt Pleasant NW', '17th & G St NW', '12th & U St NW', '14th & V St NW', 'Court House Metro / 15th & N Uhle St ', 'New Hampshire Ave & T St NW', '5th St & Massachusetts Ave NW', 'Ballston Metro / N Stuart & 9th St N', '15th & K St NW', '11th & M St NW', '1st & Rhode Island Ave NW', '1st & O St NW', 'M St & Pennsylvania Ave NW', '15th & P St NW', '14th & L St NW', '10th & K St NW', '7th & T St NW', '3rd & Elm St NW', 'Convention Center / 7th & M St NW', 'New Jersey Ave & R St NW', '15th & Euclid St  NW', '8th & O St NW', '14th & Girard St NW', 'Lincoln Park / 13th & East Capitol St NE ', '16th & Irving St NW', '3rd & H St NW', 'Henry St & Pendleton St', '18th St & Wyoming Ave NW', '6th & K St NE', '16th & Harvard St NW', '14th St & New York Ave NW

In [134]:
print(str(len(list_stations)) + " items exported for geocoding...")
list_stations_df = pd.DataFrame(list_stations, columns=['station'])
list_stations_df.to_csv('list_stations_df.csv', index = False)
#frame.to_csv('raw_frame.csv', index = False)

487 items exported for geocoding...


In [129]:
#Install Geopy Package
#import sys
#!{sys.executable} -m pip install geopy

In [130]:
# Import geocoding service
#from geopy.geocoders import GoogleV3, Nominatim, OpenMapQuest, OpenCage
#import csv

#print("CREATING GEOCODING OBJECTS...")

    ### Note that DC OPEN DATA BIKE SHARE API is used instead of geocoded address based on station address in this trip data file. See Capital Bike Share Locations API.ipynb

#### Time Band

In [140]:
print(list(frame.columns.values))

['Index', 'Duration', 'Start date', 'End date', 'Start station number', 'Start station', 'End station number', 'End station', 'Bike number', 'Member type']


In [143]:
frame.head(10)

,Index,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,1,221,2017-01-01 00:00:41,2017-01-01 00:04:23,31634,3rd & Tingey St SE,31208,M St & New Jersey Ave SE,W00869,Member
1,2,1676,2017-01-01 00:06:53,2017-01-01 00:34:49,31258,Lincoln Memorial,31270,8th & D St NW,W00894,Casual
2,3,1356,2017-01-01 00:07:10,2017-01-01 00:29:47,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,31222,New York Ave & 15th St NW,W21945,Casual
3,4,1327,2017-01-01 00:07:22,2017-01-01 00:29:30,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,31222,New York Ave & 15th St NW,W20012,Casual
4,5,1636,2017-01-01 00:07:36,2017-01-01 00:34:52,31258,Lincoln Memorial,31270,8th & D St NW,W22786,Casual
5,6,1603,2017-01-01 00:08:11,2017-01-01 00:34:55,31258,Lincoln Memorial,31270,8th & D St NW,W20890,Casual
6,7,473,2017-01-01 00:08:36,2017-01-01 00:16:29,31611,13th & H St NE,31616,3rd & H St NE,W20340,Member
7,8,200,2017-01-01 00:11:07,2017-01-01 00:14:27,31104,Adams Mill & Columbia Rd NW,31121,Calvert St & Woodley Pl NW,W20398,Member
8,9,748,2017-01-01 00:13:20,2017-01-01 00:25:49,31041,Prince St & Union St,31097,Saint Asaph & Madison St,W00365,Member
9,10,912,2017-01-01 00:14:35,2017-01-01 00:29:48,31202,14th & R St NW,31505,Eckington Pl & Q St NE,W20771,Member


In [151]:
frame['Start date']

0         2017-01-01 00:00:41
1         2017-01-01 00:06:53
2         2017-01-01 00:07:10
3         2017-01-01 00:07:22
4         2017-01-01 00:07:36
5         2017-01-01 00:08:11
6         2017-01-01 00:08:36
7         2017-01-01 00:11:07
8         2017-01-01 00:13:20
9         2017-01-01 00:14:35
10        2017-01-01 00:16:08
11        2017-01-01 00:16:08
12        2017-01-01 00:16:15
13        2017-01-01 00:16:58
14        2017-01-01 00:17:04
15        2017-01-01 00:18:13
16        2017-01-01 00:18:42
17        2017-01-01 00:18:48
18        2017-01-01 00:21:15
19        2017-01-01 00:22:13
20        2017-01-01 00:23:21
21        2017-01-01 00:23:28
22        2017-01-01 00:24:45
23        2017-01-01 00:25:20
24        2017-01-01 00:25:42
25        2017-01-01 00:25:43
26        2017-01-01 00:25:57
27        2017-01-01 00:26:09
28        2017-01-01 00:28:05
29        2017-01-01 00:28:26
                 ...         
815234    2017-12-31 22:23:52
815235    2017-12-31 22:27:14
815236    

In [158]:
#datetime string processing
frame = frame.rename(columns={"Start date": "Start_date"})
frame = frame.rename(columns={"End date": "End_date"})

In [192]:
#Further reference on datetimeindex: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.html
from datetime import datetime
import calendar
def convert_time(df,col_head):
    col_name = col_head + '_date'
    
    temp_year = col_head + '_year'
    temp_month = col_head + '_month'
    temp_day = col_head + '_day'
    temp_hour = col_head + '_hour'
    temp_minute = col_head + '_minute'
    temp_second = col_head + '_second'

    df[temp_year] = pd.DatetimeIndex(df[col_name]).year
    df[temp_month] = pd.DatetimeIndex(df[col_name]).month
    df[temp_day] = pd.DatetimeIndex(df[col_name]).day
    df[temp_hour] = pd.DatetimeIndex(df[col_name]).hour
    df[temp_minute] = pd.DatetimeIndex(df[col_name]).minute
    df[temp_second] = pd.DatetimeIndex(df[col_name]).second
    
    del col_name, temp_year, temp_month, temp_day, temp_hour, temp_minute, temp_second
    return(df)
    print(df.columns.values)

In [194]:
convert_time(frame,'Start')

,Index,Duration,Start_date,End_date,Start station number,Start station,End station number,End station,Bike number,Member type,Start_year,Start_month,Start_day,Start_hour,Start_minute,Start_second
0,1,221,2017-01-01 00:00:41,2017-01-01 00:04:23,31634,3rd & Tingey St SE,31208,M St & New Jersey Ave SE,W00869,Member,2017,1,1,0,0,41
1,2,1676,2017-01-01 00:06:53,2017-01-01 00:34:49,31258,Lincoln Memorial,31270,8th & D St NW,W00894,Casual,2017,1,1,0,6,53
2,3,1356,2017-01-01 00:07:10,2017-01-01 00:29:47,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,31222,New York Ave & 15th St NW,W21945,Casual,2017,1,1,0,7,10
3,4,1327,2017-01-01 00:07:22,2017-01-01 00:29:30,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,31222,New York Ave & 15th St NW,W20012,Casual,2017,1,1,0,7,22
4,5,1636,2017-01-01 00:07:36,2017-01-01 00:34:52,31258,Lincoln Memorial,31270,8th & D St NW,W22786,Casual,2017,1,1,0,7,36
5,6,1603,2017-01-01 00:08:11,2017-01-01 00:34:55,31258,Lincoln Memorial,31270,8th & D St NW,W20890,Casual,2017,1,1,0,8,11
6,7,473,2017-01-01 00:08:36,2017-01-01 00:16:29,31611,13th & H St NE,31616,3rd & H St NE,W20340,Member,2017,1,1,0,8,36
7,8,200,2017-01-01 00:11:07,2017-01-01 00:14:27,31104,Adams Mill & Columbia Rd NW,31121,Calvert St & Woodley Pl NW,W20398,Member,2017,1,1,0,11,7
8,9,748,2017-01-01 00:13:20,2017-01-01 00:25:49,31041,Prince St & Union St,31097,Saint Asaph & Madison St,W00365,Member,2017,1,1,0,13,20
9,10,912,2017-01-01 00:14:35,2017-01-01 00:29:48,31202,14th & R St NW,31505,Eckington Pl & Q St NE,W20771,Member,2017,1,1,0,14,35


In [195]:
convert_time(frame,'End')

,Index,Duration,Start_date,End_date,Start station number,Start station,End station number,End station,Bike number,Member type,...,Start_day,Start_hour,Start_minute,Start_second,End_year,End_month,End_day,End_hour,End_minute,End_second
0,1,221,2017-01-01 00:00:41,2017-01-01 00:04:23,31634,3rd & Tingey St SE,31208,M St & New Jersey Ave SE,W00869,Member,...,1,0,0,41,2017,1,1,0,4,23
1,2,1676,2017-01-01 00:06:53,2017-01-01 00:34:49,31258,Lincoln Memorial,31270,8th & D St NW,W00894,Casual,...,1,0,6,53,2017,1,1,0,34,49
2,3,1356,2017-01-01 00:07:10,2017-01-01 00:29:47,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,31222,New York Ave & 15th St NW,W21945,Casual,...,1,0,7,10,2017,1,1,0,29,47
3,4,1327,2017-01-01 00:07:22,2017-01-01 00:29:30,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,31222,New York Ave & 15th St NW,W20012,Casual,...,1,0,7,22,2017,1,1,0,29,30
4,5,1636,2017-01-01 00:07:36,2017-01-01 00:34:52,31258,Lincoln Memorial,31270,8th & D St NW,W22786,Casual,...,1,0,7,36,2017,1,1,0,34,52
5,6,1603,2017-01-01 00:08:11,2017-01-01 00:34:55,31258,Lincoln Memorial,31270,8th & D St NW,W20890,Casual,...,1,0,8,11,2017,1,1,0,34,55
6,7,473,2017-01-01 00:08:36,2017-01-01 00:16:29,31611,13th & H St NE,31616,3rd & H St NE,W20340,Member,...,1,0,8,36,2017,1,1,0,16,29
7,8,200,2017-01-01 00:11:07,2017-01-01 00:14:27,31104,Adams Mill & Columbia Rd NW,31121,Calvert St & Woodley Pl NW,W20398,Member,...,1,0,11,7,2017,1,1,0,14,27
8,9,748,2017-01-01 00:13:20,2017-01-01 00:25:49,31041,Prince St & Union St,31097,Saint Asaph & Madison St,W00365,Member,...,1,0,13,20,2017,1,1,0,25,49
9,10,912,2017-01-01 00:14:35,2017-01-01 00:29:48,31202,14th & R St NW,31505,Eckington Pl & Q St NE,W20771,Member,...,1,0,14,35,2017,1,1,0,29,48


In [196]:
#CONFIRM TIME WORK
frame.columns.values

array(['Index', 'Duration', 'Start_date', 'End_date',
       'Start station number', 'Start station', 'End station number',
       'End station', 'Bike number', 'Member type', 'Start_year',
       'Start_month', 'Start_day', 'Start_hour', 'Start_minute',
       'Start_second', 'End_year', 'End_month', 'End_day', 'End_hour',
       'End_minute', 'End_second'], dtype=object)

In [205]:
# FIND DAY OF WEEK / 0 Mon - 6 Sun
frame['DayWeek_S'] = pd.to_datetime(frame['Start_date']).dt.dayofweek
frame['DayWeek_E'] = pd.to_datetime(frame['End_date']).dt.dayofweek

In [210]:
# ADJUST CODE / 1 Mon - 7 Sun
frame['DayWeek_S'] = frame['DayWeek_S'] + 1
#pd.crosstab(index=frame["DayWeek_S"],columns="count")
frame['DayWeek_E'] = frame['DayWeek_E'] + 1
#pd.crosstab(index=frame["DayWeek_E"],columns="count")

In [224]:
if ((frame['DayWeek_S'].min() == 1) & (frame['DayWeek_S'].max() == 7)):
    if ((frame['DayWeek_E'].min() == 1) & (frame['DayWeek_E'].max() == 7)):
        print("Day of Week Variable Range: 1 Monday - 7 Sunday")
    else:
        print("Incomplete recoding for day of week variable")
else:
    print("Check current coding for day of week. 2")
    pd.crosstab(index=frame["DayWeek_S"],columns="count")

Day of Week Variable Range: 1 Monday - 7 Sunday


In [228]:
#SORT
frame = frame.sort_values(['DayWeek_S','DayWeek_E'], ascending=True)
frame[['DayWeek_S','DayWeek_E']].head(5)

,DayWeek_S,DayWeek_E
4063,1,1
4064,1,1
4065,1,1
4066,1,1
4067,1,1


In [241]:
#EXPORTING CSV
print(frame.shape)
frame.to_csv('Clean21Sep.csv', index = False)

(3757777, 24)


In [243]:
frame.columns.values

array(['Index', 'Duration', 'Start_date', 'End_date',
       'Start station number', 'Start station', 'End station number',
       'End station', 'Bike number', 'Member type', 'Start_year',
       'Start_month', 'Start_day', 'Start_hour', 'Start_minute',
       'Start_second', 'End_year', 'End_month', 'End_day', 'End_hour',
       'End_minute', 'End_second', 'DayWeek_S', 'DayWeek_E'], dtype=object)

### Collapse 

In [284]:
#Generate count
frame_count = frame.groupby(['Start station number','End station number','DayWeek_S','Member type','Start_hour'],as_index=False).count()
frame_collapse = frame_count[['Start station number','End station number','DayWeek_S','Member type','Start_hour','Index']]
frame_sum = frame.groupby(['Start station number','End station number','DayWeek_S','Member type','Start_hour'],as_index=False).sum()
frame_sum = frame_sum[['Start station number','End station number','DayWeek_S','Member type','Start_hour','Duration']]
frame_collapse = pd.merge(frame_collapse,frame_sum, on=['Start station number','End station number','DayWeek_S','Member type','Start_hour'])
del frame_count, frame_sum
frame_collapse.rename(columns={'Index':'Trips'},inplace=True)
frame_collapse.head(5)

,Start station number,End station number,DayWeek_S,Member type,Start_hour,Trips,Duration
0,31000,31000,1,Casual,7,3,21005
1,31000,31000,1,Casual,9,2,11139
2,31000,31000,1,Casual,11,1,1369
3,31000,31000,1,Casual,12,1,4031
4,31000,31000,1,Casual,13,1,2372


In [285]:
#EXPORTING CSV
print(frame_collapse.shape)
frame_collapse.to_csv('Clean_Collapse.csv', index = False)

(1580428, 7)


### Preliminary Exploratory Data Analysis

In [291]:
frame_collapseEDA = frame_collapse.groupby(['DayWeek_S','Member type','Start_hour'],as_index=False).mean()
frame_collapseEDA = frame_collapseEDA[['DayWeek_S','Member type','Start_hour','Trips','Duration']]
frame_collapseEDA.head(5)

,DayWeek_S,Member type,Start_hour,Trips,Duration
0,1,Casual,0,1.938567,4500.943686
1,1,Casual,1,1.742765,3808.080386
2,1,Casual,2,1.690909,3146.303030
3,1,Casual,3,1.468354,3081.835443
4,1,Casual,4,1.380282,3454.774648


In [339]:
frame_collapseEDA.shape

(336, 5)

In [400]:
#Function define:
    #1. Slice by day
    #2. Preserve one outcome column, as defined by colname
def dayslice_S(df,colname,member='Member'):
    Days = ['Mon','Tue','Wed','Thur','Fri','Sat','Sun']
    output = {}
    i = 0
    for d in Days:
        i = i + 1
        dfname = d + '_' + colname
        output[dfname] = pd.DataFrame(df.loc[(df['DayWeek_S']==i) & (df['Member type']==member)])[['Start_hour',colname]]
    del Days, i, dfname
    return(output)

In [420]:
Trips_M = dayslice_S(frame_collapseEDA,'Trips','Member')
Trips_C = dayslice_S(frame_collapseEDA,'Trips','Casual')
Duration_M = dayslice_S(frame_collapseEDA,'Duration','Member')
Duration_C = dayslice_S(frame_collapseEDA,'Duration','Casual')

In [326]:
#Import new package
    #install plotly in environment
    #! pip install plotly
import plotly.plotly as py
import plotly.graph_objs as go
    #Already imported
    #from datetime import datetime

from matplotlib import pyplot

from plotly.offline import iplot, init_notebook_mode
init_notebook_mode()

In [417]:
Mon_TM = go.Scatter(
            x=pd.DataFrame(Trips_M['Mon_Trips']).Start_hour,
            y=pd.DataFrame(Trips_M['Mon_Trips'])['Trips'],
            name = "Monday Trips - Member",
            line = dict(color = '#17BECF'),
            opacity = 0.8)
Mon_TC = go.Scatter(
            x=pd.DataFrame(Trips_C['Mon_Trips']).Start_hour,
            y=pd.DataFrame(Trips_C['Mon_Trips'])['Trips'],
            name = "Monday Trips - Casual",
            line = dict(color = '#7F7F7F'),
            opacity = 0.8)

In [421]:
data = [Mon_TM,Mon_TC]
layout = dict(
    title = "Trips Starting Time",
    xaxis = dict(
        range = ['00Hours','23Hours'])
)

In [422]:
fig = dict(data=data, layout=layout)
iplot(fig, filename = "Monday Trips Starting Time")